In [86]:
import pandas as pd
import numpy as np
import itertools
import sys
sys.path.insert(0,'../..')

import matplotlib.pyplot as plt

from g4l.data import Sample
from g4l.tree import ContextTree

#cache_dir = '../example1/cache'

In [87]:
# Create a sample object instance
X = Sample('../example1/folha.txt', [0, 1, 2, 3, 4])
t = ContextTree(X, max_depth=4)

In [114]:
df = t.df.copy()
df['leaf'] = 0
max_node_length = df.l.max() # 4

# Eliminate nodes with no contribution
df = df[df.lps<0]

# Mark all last-level nodes as leaf
df.loc[df.l==max_node_length, 'leaf'] = 1

In [123]:
leaves = df[df.leaf==1]
#for parent_idx in leaves.parent_idx.unique():
parent_idx = leaves.parent_idx.unique()[0]
sum_likelihood = df[df.parent_idx==parent_idx].lps.sum()
df.loc[df.node_idx==parent_idx, 'transition_sum_log_probs'] = sum_likelihood
df.loc[df.node_idx==parent_idx]

,l,node_idx,len_idx,parent_idx,node,node_freq,lps,transition_probs,transition_sum_log_probs,flag,final,leaf
30,3,30,0,5,000,3303,-1975.429579,"[0.28549803209203756, 0.7145019679079625, 0.0,...",-1952.18,0,0,0


In [ ]:
df2 = pd.DataFrame(columns=['num_contexts', 'likelihood', 'tree'])

while True:
    # get all leaf nodes
    leaves = df[df.leaf==1]
    parents_idx = leaves.parent_idx.unique()
    
    
    
    df.sort_values(['transition_sum_log_probs'], ascending=False)
    for j, row in df.iterrows():
    df.loc[df.node_idx==row.node_idx, 'flag'] = removal_order
    removal_order += 1
    likelihood = df[df.flag==0].lps.sum()
    df2.loc[len(df2)] = [1, likelihood]
df2

#parents = df.parent_idx.unique()
#leaves = df[df.parent_idx==parents[2]]
#children_likelihood_sum = leaves.lps.sum()
#print(children_likelihood_sum)
#leaves

In [83]:
t.df[t.df.node_idx==parents[2]]

,l,node_idx,len_idx,parent_idx,node,node_freq,lps,transition_probs,transition_sum_log_probs,flag,final
145,3,145,115,28,034,19,-12.50409,"[0.0, 0.0, 0.631578947368421, 0.36842105263157...",-1.458061,0,0


In [51]:
# TODO: rename child_probs to next_symbol_transition_prob
x = df[df.node.str.match('.334')]
x

,l,node_idx,len_idx,parent_idx,node,node_freq,lps,transition_probs,transition_sum_log_probs,flag,final
745,4,745,590,148,0334,2,-1.386294,"[0.0, 0.0, 0.5, 0.5, 0.0]",-1.386294,0,0


In [50]:
x['clc'] = x.node_freq * x.lps
x.head(5)

/home/arthur/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,l,node_idx,len_idx,parent_idx,node,node_freq,lps,transition_probs,transition_sum_log_probs,flag,final,clc
745,4,745,590,148,0334,2,-1.386294,"[0.0, 0.0, 0.5, 0.5, 0.0]",-1.386294,0,0,-2.77259
746,4,746,591,148,1334,0,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0,0,0
747,4,747,592,148,2334,0,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0,0,0
748,4,748,593,148,3334,1,0.000000,"[0.0, 0.0, 1.0, 0.0, 0.0]",0.000000,0,0,0
749,4,749,594,148,4334,0,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0,0,0


In [57]:
t.df.lps.sum()

-339854.51291223057

array([148, 103, 145, 124, 128, 33, 49, 146, 69, 137, 123, 44, 98, 133,
       120, 99, 130, 43, 126, 121, 112, 95, 48, 125, 101, 100, 68, 105,
       87, 96, 135, 108, 45, 83, 80, 32, 65, 131, 37, 110, 57, 31, 40, 46,
       30, 66, 41, 85, 106, 55, 81, 35], dtype=object)

In [ ]:
x = t.df
x = x.sort_values(['node_freq'], ascending=False)
x = x.groupby(['node_freq']).median()
plt.figure(figsize=(14, 6))
plt.plot(x.lps)
#plt.xscale('log')
plt.show()
x.index